In [2]:
import os
import json
import shutil

import pandas as pd
pd.options.mode.chained_assignment = None
import matplotlib.pyplot as plt
import numpy as np
#from mplfinance.original_flavor import candlestick2_ohlc
from tqdm import tqdm

In [20]:
class test():
    def __init__(self, reward_function):
        if hasattr(self, reward_function):
            self.function = getattr(self, reward_function)
        else:
            self.function = self.a
        
    def a(self, t):
        print("a")
    def b(self,t):
        print("b")

In [25]:
tmp = test("b")

In [26]:
tmp.function(1)

b


In [13]:
'Perp_BTC_FundingRate_Data_fakehist' in 'Perp_BTC_FundingRate_Data_fakehist.csv'

True

# Overview of Datasets columns

In [5]:
btc_df = pd.read_csv('./../data/BTC_histData_dt1800.0s_20220825_0629.csv', sep=',')
# btc_df = pd.read_csv('./../data/Perp_BTC_FundingRate_Data.csv', sep=',')

print(btc_df.shape)
print(btc_df.columns)
btc_df.head()

(44665, 26)
Index(['date', 'open', 'high', 'low', 'close', 'Volume', '20ema', '12ema',
       '26ema', '50ema', '200ema', 'mfi', 'obv_val', 'obv_ema', 'stochRsi13',
       'stochRsi_d', 'stochRsi_k', 'rsi14', 'rsi5', 'macd', 'signal',
       'histogram', 'macd2', 'signal2', 'hist2', 'label'],
      dtype='object')


,date,open,high,low,close,Volume,20ema,12ema,26ema,50ema,...,stochRsi_k,rsi14,rsi5,macd,signal,histogram,macd2,signal2,hist2,label
0,2020-01-01 00:00:00,7195.24,7196.25,7175.47,7178.45,331.185522,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-
1,2020-01-01 00:30:00,7178.19,7185.44,7175.46,7177.02,180.629379,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-
2,2020-01-01 01:00:00,7176.47,7217.00,7175.71,7212.10,355.063574,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-
3,2020-01-01 01:30:00,7212.10,7230.00,7204.28,7216.27,527.989029,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-
4,2020-01-01 02:00:00,7215.52,7238.88,7211.41,7228.09,309.018157,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-


In [5]:
btc_df.histogram.describe()

count    44629.000000
mean        -0.003988
std         82.429895
min       -653.868219
25%        -26.479130
50%         -0.006819
75%         25.202251
max        592.580710
Name: histogram, dtype: float64

In [3]:
swap_df = pd.read_csv('./../data/Perp_BTC_FundingRate_Data.csv', sep=',')
swap_df.rename({'EMA50': '50ema', 'RSI14': 'rsi14', 'MACD': 'macd', 'Open':'open', 'Close':'close'}, axis=1, inplace=True)

In [6]:
swap_df['histogram']=btc_df['histogram']

In [7]:
swap_df.to_csv('./../data/Perp_BTC_FundingRate_Data_fakehist.csv', sep=',')

In [8]:
print(swap_df.shape)
print(swap_df.columns)
swap_df.head()

(61725, 19)
Index(['Unnamed: 0', 'Open time', 'Open_BTC', 'Close_BTC', 'open', 'High',
       'Low', 'close', 'Volume', 'Close time', 'Quote asset volume',
       'Number of trades', 'Taker buy base asset volume',
       'Taker buy quote asset volume', '50ema', 'rsi14', 'macd',
       'Funding_Rate', 'histogram'],
      dtype='object')


,Unnamed: 0,Open time,Open_BTC,Close_BTC,open,High,Low,close,Volume,Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,50ema,rsi14,macd,Funding_Rate,histogram
0,0,2019-09-10 10:00:00,10257.03,10245.15,10255.07,10280.90,10248.07,10250.88,179.866,1568111399999,1.846115e+06,165,166.588,1.709805e+06,NaN,NaN,NaN,0.010000%,NaN
1,1,2019-09-10 10:30:00,10246.75,10239.92,10247.21,10255.87,10222.99,10240.32,178.447,1568113199999,1.827575e+06,154,169.928,1.740321e+06,NaN,NaN,NaN,0.010000%,NaN
2,2,2019-09-10 11:00:00,10240.14,10231.53,10240.11,10257.66,10224.36,10229.00,189.144,1568114999999,1.936715e+06,155,174.225,1.784091e+06,NaN,NaN,NaN,0.010000%,NaN
3,3,2019-09-10 11:30:00,10229.37,10252.38,10232.26,10259.52,10223.56,10254.56,130.340,1568116799999,1.335149e+06,145,87.989,9.014285e+05,NaN,NaN,NaN,0.010000%,NaN
4,4,2019-09-10 12:00:00,10251.22,10233.20,10252.70,10252.70,10226.02,10234.14,151.801,1568118599999,1.553913e+06,159,107.057,1.095882e+06,NaN,NaN,NaN,0.010000%,NaN


In [26]:
float(swap_df.Funding_Rate[0].replace("%", ""))/100

0.0001

In [31]:
print(btc_df['label'].unique())
btc_df['label'].describe()

['-' 'strong_buy' 'hold' 'sell' 'buy']


count     44665
unique        5
top           -
freq      29846
Name: label, dtype: object

In [6]:
pd.read_csv('./../data/20220828/Epi_data.csv', sep=',').columns

Index(['Unnamed: 0', 'episode', '#buy_actions', '#sell_actions', 'money',
       'fee', 'profit', 'epsilon'],
      dtype='object')

# Dataset used for training on one episode

In [9]:
act_df = pd.read_csv('./../data/20220828/action_data_e1.csv', sep=',')
print(act_df.shape)
print(act_df.columns)
act_df.head()

(6710, 13)
Index(['Unnamed: 0', 'episode', 'run', 'timestep', 'date', 'action', 'state',
       'money_free', 'money_fiktiv', 'invest', 'fee', 'reward', 'profit'],
      dtype='object')


,Unnamed: 0,episode,run,timestep,date,action,state,money_free,money_fiktiv,invest,fee,reward,profit
0,0,1,1,0,2020-03-29 04:00:00,hold,"[0.5, 0.9053371894725785, 0.5, 0.5, 0.46806793...",200.0,200.00,0.0,0.0,0.00,0.0
1,0,1,1,1,2020-03-29 04:30:00,buy_100,"[0.5, 0.9053371894725785, 0.5, 0.5, 0.46806793...",0.0,199.80,199.8,0.2,-0.20,0.0
2,0,1,1,2,2020-03-29 05:00:00,hold,"[0.5, 0.9053371894725785, 0.5, 0.5, 0.46806793...",0.0,199.06,199.8,0.0,-0.94,0.0
3,0,1,1,3,2020-03-29 05:30:00,hold,"[0.5, 0.9053371894725785, 0.5, 0.5, 0.46806793...",0.0,197.99,199.8,0.0,-2.01,0.0
4,0,1,1,4,2020-03-29 06:00:00,hold,"[0.5, 0.9053371894725785, 0.5, 0.5, 0.46806793...",0.0,198.55,199.8,0.0,-1.45,0.0


## States

States are build of windowed values from:
 - close values of BTC
 - histogram
 - 50ema
 - rsi14
 - Money in Wallet.

In [20]:
states = list()
for ele in act_df['state']:
    states.append(json.loads(ele))
states = np.array(states)

In [24]:
states[20]

array([9.83209352e-01, 9.05337189e-01, 1.09432023e-01, 5.00000000e-01,
       4.68067937e-01, 1.04691834e-10, 8.96768986e-01, 5.13919347e-02,
       5.00000000e-01, 4.76010250e-01, 3.41704586e-15, 6.34299082e-01,
       1.61833744e-02, 4.50166003e-01, 4.36390945e-01, 9.99999975e-01,
       6.33411731e-02, 3.69794734e-02, 3.22962406e-01, 3.86211583e-01,
       1.11953595e-05, 3.08678275e-02, 2.71477320e-02, 2.54847711e-01,
       4.23724088e-01, 9.96731010e-01, 1.14207001e-02, 3.86320688e-02,
       6.37377781e-01, 4.06313170e-01, 9.56531867e-03, 1.16444019e-02,
       3.66082935e-02, 4.09218615e-01, 4.19208563e-01, 6.24450489e-20,
       1.17904044e-02, 7.56927170e-03, 5.45930762e-01, 4.11402171e-01,
       1.00000000e+00, 8.36825856e-04, 2.20130274e-02, 4.62705538e-01,
       3.45391095e-01, 2.74747063e-08, 1.04156103e-03, 1.30243059e-02,
       1.93982741e-01, 3.98769490e-01, 2.74840602e-12, 5.82990193e-04,
       5.47522884e-03, 6.75203703e-01, 3.73627613e-01, 9.99999999e-01,
      

## Actions

In [26]:
act_df['action'].unique()

array(['hold', 'buy_100', 'sell', 'buy_50'], dtype=object)

In [27]:
act_df['action'].describe()

count     6710
unique       4
top       hold
freq      6425
Name: action, dtype: object